In [1]:
# Decorations
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gmplot
from config import api_key
from pprint import pprint

In [2]:
# Import database
df1 = pd.read_csv('Data/housing.csv')
df2 = pd.read_csv('Data/Total.csv')
df1 = df1.dropna()

### df1(1990) Analysis

Sort and slice data to get top and bottom 5%

In [ ]:
# SSort data by house value
df1 = df1.sort_values('median_house_value', ascending=False).reset_index(drop=True)
df1.head()

In [ ]:
# Slice the top 5%
top5 = df1.iloc[:round(len(df1['median_house_value'])*0.05), :].sort_values('median_house_value', ascending=True)

In [ ]:
# slice the bottom 5%
bottom5 = df1.iloc[round(len(df1['median_house_value'])*0.95): len(df1['median_house_value']),:].sort_values('median_house_value', ascending=True)

Plotting Top 5% and Bottom 5% on gmap (Use gmplot package)

In [ ]:
# Set the center of the map
gmap = gmplot.GoogleMapPlotter(top5['latitude'].median(),
                                   top5['longitude'].median(), 100000)
# Plot scatter points based on LatLng
gmap.scatter(top5['latitude'], top5['longitude'], '#FF0000', 
                              size = 2000, marker = False ) 
gmap.scatter(bottom5['latitude'], bottom5['longitude'], '#110870', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap.draw("Plot/gmap.html")

### df2(2018) Analysis 

In [3]:
# Keep only the columns that are useful 
df2 = df2[['Sub Type', 'St#', 'St Name', 'City', 'L/C Price', 'Br/Ba', 'YrBuilt']]
df2.head()
#df2 = df2.drop(on=0)

,Sub Type,St#,St Name,City,L/C Price,Br/Ba,YrBuilt
0,MANL/D,12,Via Veneto #12,CHU,"$89,000","2/2,0,0,0",2006
1,CONDO/A,1653,Pentecost Way #1,SD,"$158,000","1/1,0,0,0",1980
2,CONDO/A,6101,Adelaide Ave #107,SD,"$168,250","0/1,0,0,0",1984/ASR
3,CONDO/A,589 N,Johnson #137,ELC,"$175,000","2/1,0,0,0",1980
4,SFR,3138,Roadrunner,BSPG,"$190,000","2/2,0,0,0",1986


In [4]:
yr_blt = df2.loc[:, 'YrBuilt'].str.split('/', expand=True)[0]
yr_blt = pd.DataFrame(yr_blt)
yr_blt = yr_blt.fillna(0)

In [5]:
df2['Age'] = ''
count = 0
for i in yr_blt[0]:
        i = 2018 - int(i)
        df2['Age'][count] = i
        count += 1
df2.head()

,Sub Type,St#,St Name,City,L/C Price,Br/Ba,YrBuilt,Age
0,MANL/D,12,Via Veneto #12,CHU,"$89,000","2/2,0,0,0",2006,12
1,CONDO/A,1653,Pentecost Way #1,SD,"$158,000","1/1,0,0,0",1980,38
2,CONDO/A,6101,Adelaide Ave #107,SD,"$168,250","0/1,0,0,0",1984/ASR,34
3,CONDO/A,589 N,Johnson #137,ELC,"$175,000","2/1,0,0,0",1980,38
4,SFR,3138,Roadrunner,BSPG,"$190,000","2/2,0,0,0",1986,32


In [6]:
df2['house_price'] = df2.loc[:, 'L/C Price'].str.split('$', expand=True)[1] 

In [7]:
df2.head()

,Sub Type,St#,St Name,City,L/C Price,Br/Ba,YrBuilt,Age,house_price
0,MANL/D,12,Via Veneto #12,CHU,"$89,000","2/2,0,0,0",2006,12,"89,000"
1,CONDO/A,1653,Pentecost Way #1,SD,"$158,000","1/1,0,0,0",1980,38,"158,000"
2,CONDO/A,6101,Adelaide Ave #107,SD,"$168,250","0/1,0,0,0",1984/ASR,34,"168,250"
3,CONDO/A,589 N,Johnson #137,ELC,"$175,000","2/1,0,0,0",1980,38,"175,000"
4,SFR,3138,Roadrunner,BSPG,"$190,000","2/2,0,0,0",1986,32,"190,000"


In [8]:
df2_by_value = df2.set_index("L/C Price")
df2_by_value = df2_by_value.reset_index()
df2_by_value.head()

,L/C Price,Sub Type,St#,St Name,City,Br/Ba,YrBuilt,Age,house_price
0,"$89,000",MANL/D,12,Via Veneto #12,CHU,"2/2,0,0,0",2006,12,"89,000"
1,"$158,000",CONDO/A,1653,Pentecost Way #1,SD,"1/1,0,0,0",1980,38,"158,000"
2,"$168,250",CONDO/A,6101,Adelaide Ave #107,SD,"0/1,0,0,0",1984/ASR,34,"168,250"
3,"$175,000",CONDO/A,589 N,Johnson #137,ELC,"2/1,0,0,0",1980,38,"175,000"
4,"$190,000",SFR,3138,Roadrunner,BSPG,"2/2,0,0,0",1986,32,"190,000"


### Getting LatLng using geocoder API for df2

In [17]:
# import CityCode
citycode_df = pd.read_csv('Data/CleanCityCode_nospace.csv')
citycode_df = citycode_df[['City', 'Code']]
citycode_df['Code'] = citycode_df['Code'].str.strip()

In [25]:
df2_by_value['City Name'] = ""
for index, row in df2_by_value.iterrows():
    for i, r in citycode_df.iterrows():
        if r['Code'] == row['City']:
            row['City Name'] = r['City']
            if index%50 == 0:
                print(index)

0
50
100
150
150
200
250
300
300
350
450
500
550
600
650
700
750
800
850
900
1050
1100
1150
1200
1250
1300
1300
1350
1450
1500
1550
1600
1650
1700
1750
1800
1900
2050
2100
2150
2150
2200
2250
2300
2400
2450
2550
2600
2650
2700
2750
2800
2850
2900
2900
2950
3000
3050
3100
3150
3150
3200
3250
3250
3250
3300
3350
3350
3400
3450
3550
3600
3650
3700
3750
3750
3750
3850
3900
3950
4050
4100
4150
4150
4200
4250
4300
4350
4400
4550
4600
4650
4700
4750
4800
4800
4850
4900


In [28]:
df2_by_value.groupby('City Name').count()

,L/C Price,Sub Type,St#,St Name,City,Br/Ba,YrBuilt,Age,house_price
City Name,,,,,,,,,
,939,939,939,939,937,939,935,939,939
29 Palms,8,8,8,8,8,8,8,8,8
Adelanto,6,6,6,6,6,6,6,6,6
Agoura,1,1,1,1,1,1,1,1,1
Agoura Hills,7,7,7,7,7,7,7,7,7
Aguanga,1,1,1,1,1,1,1,1,1
Alhambra,10,10,10,10,10,10,10,10,10
Aliso Viejo,15,15,15,15,15,15,15,15,15
Alpine,5,5,5,5,5,5,5,5,5


In [ ]:
df2_by_value["Address"] = df2_by_value["St#"].astype(str) + " " + df2_by_value["St Name"].astype(str) + " " + df2_by_value['City']
df2_by_value['Lat'] = ""
df2_by_value['Lng'] = ""

In [ ]:
params = {"key": api_key}
for index, row in df2_by_value.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params['address'] = row['Address']
    geo_data = requests.get(base_url, params).json()
    try:
        df2_by_value["Lat"][index] = geo_data["results"][0]["geometry"]["location"]["lat"]
        df2_by_value["Lng"][index] = geo_data["results"][0]["geometry"]["location"]["lng"]
    except IndexError:
        print(f"Row {index} cannot be found on gmap.")
        continue

In [ ]:
# Output to CSV
df2_by_value.to_csv("Data/df2_latlng.csv")

### Plotting df2_latlng

In [ ]:
df2_latlng = pd.read_csv("Data/df2_latlng.csv")

In [ ]:
df2_latlng

In [ ]:
# Set the center of the map
gmap_df2 = gmplot.GoogleMapPlotter(df2_latlng['Lat'].median(),
                                df2_latlng['Lng'].median(), 100000)
# Plot scatter points based on LatLng
gmap_df2.scatter(df2_latlng['Lat'], df2_latlng['Lng'], '#FF0000', 
                              size = 10000, marker = False) 
# Draw out to 'Plot' folder in html format
gmap_df2.draw("Plot/gmap_df2.html")

## df1 vs df2